In [1]:
import sys
sys.path.append("D:\\vnpy\\vnpy2_pro")

In [2]:
from datetime import datetime

import pandas as pd

from vnpy.trader.database import database_manager
from vnpy.trader.constant import Exchange, Interval

data = database_manager.load_bar_data(
    symbol="AG99",
    exchange=Exchange.SHFE,
    interval=Interval.DAILY,
    start=datetime(2020,1,1),
    end=datetime(2020,12,12)
)

In [15]:
# 涨跌幅度统计
up_list = []
down_list = []
range_list = []

count = 0
for bar_data in data:
    count += 1
    range_list.append(bar_data.high_price - bar_data.low_price)
    up_list.append(bar_data.high_price - bar_data.open_price)
    down_list.append(bar_data.open_price - bar_data.low_price)
    

up_list = sorted(up_list)
down_list = sorted(down_list)
range_list = sorted(range_list)

up_count = 0
for item in up_list:
    if item > 16:
        up_count += 1
        
down_count = 0
for item in down_list:
    if item > 16:
        down_count += 1
        
print(up_count/count)
print(down_count/count)

# print(up_list)
# print(down_list)
# print(range_list)

0.7168674698795181
0.6746987951807228


In [3]:
# 连涨连跌统计
data_dict = {}
bar_count = 0

lock = 0
count = 0

for bar_data in data:
    bar_count += 1
    
    if bar_data.close_price > bar_data.open_price:
        if lock == 0:
            count = 1
            lock = 1
        elif lock == 1:
            count += 1
        elif  lock == -1:
            data_dict.setdefault(str(count), 0)
            data_dict[str(count)] += 1
            
            lock = 1
            count = 1
            
        
        
    elif bar_data.close_price < bar_data.open_price:
        if lock == 0:
            count = 1
            lock = -1
        elif lock == -1:
            count += 1
        elif lock == 1:
            data_dict.setdefault(str(count), 0)
            data_dict[str(count)] += 1
            
            lock = -1
            count = 1
        
    else:
        up_or_down = 0
        count = 0

print(bar_count)
print(data_dict)



166
{'1': 47, '2': 19, '3': 9, '4': 5, '8': 2, '6': 2, '5': 1}


In [4]:
# 周交易日统计
weekday_dict = {
            "0": {"count": 0, "up": 0, "down": 0, "flat":0},
            "1": {"count": 0, "up": 0, "down": 0, "flat":0},
            "2": {"count": 0, "up": 0, "down": 0, "flat":0},
            "3": {"count": 0, "up": 0, "down": 0, "flat":0},
            "4": {"count": 0, "up": 0, "down": 0, "flat":0},
}

for bar_data in data:
    weekday = bar_data.datetime.weekday()
    
    weekday_dict[str(weekday)]["count"] += 1
    if bar_data.close_price > bar_data.open_price:
        weekday_dict[str(weekday)]["up"] += 1
    elif bar_data.close_price < bar_data.open_price:
        weekday_dict[str(weekday)]["down"] += 1
    else:
        weekday_dict[str(weekday)]["flat"] += 1

for key, item in weekday_dict.items():
    print(f"星期{int(key) + 1}  up : {round(item['up']/item['count'], 2)}  down : {round(item['down']/item['count'], 2)}")
#     print(item)

星期1  up : 0.52  down : 0.48
星期2  up : 0.55  down : 0.45
星期3  up : 0.59  down : 0.41
星期4  up : 0.44  down : 0.56
星期5  up : 0.53  down : 0.47
